In [4]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    KubernetesOnlineEndpoint,
    KubernetesOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential


In [2]:
# enter details of your AML workspace
subscription_id = "ea6d544d-2425-4667-8d38-51f050d9d69e"
resource_group = "rg-vtpoc-9dev"
workspace = "mlw-vtpoc-9dev"


In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)


In [5]:
# Define an endpoint name
endpoint_name = "my-endpoint"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=endpoint_name,
    description="this is a sample online endpoint",
)
print(endpoint_name)

endpt-02282058598660


In [6]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (endpt-02282058598660) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-02282058598660', 'description': 'this is a sample online endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/.azureml/inferencing/endpt-02282058598660'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x119b46010>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [25]:
model = Model(path="./models/vt-model/model.pkl")
env = Environment(
    conda_file="../data-science/environment/train-conda.yml",
    image="crvtpoc9dev.azurecr.io/local_deploy_img:v1",
)

In [26]:
blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../mlops/scripts", scoring_script="score.py"
    ),
    instance_count=1,
)

In [27]:

ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Updating local deployment (endpt-02282058598660 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM crvtpoc9dev.azurecr.io/local_deploy_img:v1
 ---> 847acc328956
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> b62640db02bf
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> f364f9adf4cd
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 78d12a42a069
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> 59fbb4e596fe
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> c0ab499e1a36
Successfully built c0ab499e1a36
Successfully tagged endpt-02282058598660:blue

Starting up endpoint.....Done (0m 30s)


KubernetesOnlineDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-02282058598660', 'type': 'Kubernetes', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x11c8787d0>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '2fbc54b506f55723f0d238d0ea428147', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x11c8b2b50>, 'version': '1', 'latest_version': None, 'path': '/Users/amohajerani/De

In [28]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32781/score', 'openapi_uri': None, 'name': 'endpt-02282058598660', 'description': 'this is a sample online endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x11c626650>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [29]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=50
)

'2024-02-29T02:49:34,258314294+00:00 - rsyslog/run \r\n2024-02-29T02:49:34,296261961+00:00 - nginx/run \r\n2024-02-29T02:49:34,329333545+00:00 - gunicorn/run \r\n2024-02-29T02:49:34,365882628+00:00 | gunicorn/run | \r\n2024-02-29T02:49:34,399351586+00:00 | gunicorn/run | ###############################################\r\n2024-02-29T02:49:34,432747336+00:00 | gunicorn/run | AzureML Container Runtime Information\r\n2024-02-29T02:49:34,466388420+00:00 | gunicorn/run | ###############################################\r\n2024-02-29T02:49:34,499238336+00:00 | gunicorn/run | \r\n2024-02-29T02:49:36,697028754+00:00 | gunicorn/run | \r\n2024-02-29T02:49:36,754937837+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20230227.v3\r\n2024-02-29T02:49:36,786541712+00:00 | gunicorn/run | \r\n2024-02-29T02:49:36,819494004+00:00 | gunicorn/run | \r\n2024-02-29T02:49:36,850632129+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/inf-conda

In [30]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../data/test-request.json",
    local=True,
)

'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body bgcolor="white">\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'